In [20]:
import numpy as np
from sklearn.neighbors import NearestNeighbors
import pickle
import json

In [21]:
def load_embeddings(path, method=None):
    """
    Load embeddings and card names from saved files
    
    Args:
        path: Path to the saved file
        method: 'pickle', 'numpy' or 'json'. If None, inferred from file extension
        
    Returns:
        embeddings: NumPy array of embeddings
        names: List of card names
    """
    if method is None:
        # Infer method from file extension
        if path.endswith('.pkl'):
            method = 'pickle'
        elif path.endswith('.npy'):
            method = 'numpy'
        elif path.endswith('.json'):
            method = 'json'
        else:
            raise ValueError("Could not infer method from file extension. Please specify method.")
    
    if method == 'pickle':
        with open(path, 'rb') as f:
            data = pickle.load(f)
        return data['embeddings'], data['names']
    
    elif method == 'numpy':
        # If path is for embeddings, expect names path to be similar
        if 'embeddings' in path:
            names_path = path.replace('embeddings', 'names')
            embeddings = np.load(path)
            with open(names_path, 'rb') as f:
                names = pickle.load(f)
            return embeddings, names
        else:
            raise ValueError("For 'numpy' method, the path should point to the embeddings file")
        
    elif method == 'json':
        with open(path, 'r') as f:
            data = json.load(f)
        return np.array(data['embeddings']), data['names']
    
    else:
        raise ValueError("Method must be 'pickle', 'numpy' or 'json'")

# Example usage:
embeddings, names = load_embeddings("./saved_embeddings/mtg_embeddings_20250724_125801.json")


In [22]:
# Apply KNN for recommendations
from sklearn.neighbors import NearestNeighbors

knn = NearestNeighbors(n_neighbors=6)  # Include the card itself + 5 recommendations
knn.fit(embeddings)

# Example: Get recommendations for a specific card
def get_recommendations(card_name, all_names, all_embeddings, knn, n=10):
    if card_name not in all_names:
        print(f"Card '{card_name}' not found in dataset.")
        return
        
    idx = all_names.index(card_name)
    distances, indices = knn.kneighbors([all_embeddings[idx]], n_neighbors=n+1)
    
    print(f"Recommendations for '{card_name}':")
    for i, idx in enumerate(indices[0][1:]):  # Skip the first one (the card itself)
        print(f"{i+1}. {all_names[idx]}")


In [23]:
get_recommendations("Rocco, Street Chef", names, embeddings, knn)

Recommendations for 'Rocco, Street Chef':
1. Visions of Phyrexia
2. Lorehold Excavation
3. Fisher's Talent
4. Oko, the Ringleader
5. Journey to the Lost City
6. Crawling Infestation
7. Evidence Examiner
8. The Book of Vile Darkness
9. Crawling Sensation
10. Doom Foretold
